In [1]:
import cvxpy as cp
import numpy as np

# Definir as matrizes constantes do sistema
M0 = np.array([
    [-2.4, -0.6, -1.7,  3.1],
    [ 0.7, -2.1, -2.6, -3.6],
    [ 0.5,  2.4, -5.0, -1.6],
    [-0.6,  2.9, -2.0, -0.6]
])

M1 = np.array([
    [ 1.1, -0.6, -0.3, -0.1],
    [-0.8,  0.2, -1.1,  2.8],
    [-1.9,  0.8, -1.1,  2.0],
    [-2.4, -3.1, -3.7, -0.1]
])

M2 = np.array([
    [ 0.9,  3.4,  1.7,  1.5],
    [-3.4, -1.4,  1.3,  1.4],
    [ 1.1,  2.0, -1.5, -3.4],
    [-0.4,  0.5,  2.3,  1.5]
])

M3 = np.array([
    [-1.0, -1.4, -0.7, -0.7],
    [ 2.1,  0.6, -0.1, -2.1],
    [ 0.4, -1.4,  1.3,  0.7],
    [ 1.5,  0.9,  0.4, -0.5]
])

# Parâmetros para a busca binária (bisseção)
rho_low = 0.0
rho_high = 5.0  # Um chute inicial para o limite superior
tolerance = 1e-5
max_iter = 100

n = M0.shape[0]
best_rho = 0.0

print("Iniciando a busca binária para encontrar o maior ρ...")
print("-" * 60)

for i in range(max_iter):
    rho = (rho_low + rho_high) / 2
    if rho_high - rho_low < tolerance:
        break

    # 1. Armazenar os vértices do sistema para o ρ atual
    A_vertices = [
        M0 + rho * M1,
        M0 + rho * M2,
        M0 + rho * M3
    ]
    A1, A2, A3 = A_vertices

    # 2. Definir as variáveis de otimização (matrizes de Lyapunov)
    P1 = cp.Variable((n, n), symmetric=True)
    P2 = cp.Variable((n, n), symmetric=True)
    P3 = cp.Variable((n, n), symmetric=True)
    
    # 3. Definir as restrições LMI para a viabilidade
    # Usamos um pequeno epsilon para garantir desigualdade estrita
    epsilon = 1e-7
    I = np.eye(n)
    
    constraints = [
        P1 >> epsilon * I,
        P2 >> epsilon * I,
        P3 >> epsilon * I,
        
        # Desigualdades de Lyapunov para cada vértice
        A1.T @ P1 + P1 @ A1 << -epsilon * I,
        A2.T @ P2 + P2 @ A2 << -epsilon * I,
        A3.T @ P3 + P3 @ A3 << -epsilon * I,
        
        # Desigualdades de Lyapunov mistas para cada par de vértices
        A1.T @ P2 + P2 @ A1 + A2.T @ P1 + P1 @ A2 << -epsilon * I,
        A1.T @ P3 + P3 @ A1 + A3.T @ P1 + P1 @ A3 << -epsilon * I,
        A2.T @ P3 + P3 @ A2 + A3.T @ P2 + P2 @ A3 << -epsilon * I,
    ]
    
    # 4. Criar e resolver o problema de viabilidade
    problem = cp.Problem(cp.Minimize(0), constraints)
    
    try:
        # Usamos um solver robusto para problemas SDP
        problem.solve(solver=cp.SCS, eps=1e-8, verbose=False)
    except cp.SolverError:
        problem.status = cp.INFEASIBLE

    # 5. Atualizar o intervalo da busca
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        print(f"Iter {i+1:2d}: Viável para ρ = {rho:.6f}. Limite inferior atualizado.")
        best_rho = rho
        rho_low = rho
    else:
        print(f"Iter {i+1:2d}: Inviável para ρ = {rho:.6f}. Limite superior atualizado.")
        rho_high = rho

print("-" * 60)
print(f"Busca finalizada após {i+1} iterações.")
print(f"\nO maior valor de ρ para estabilidade robusta é aproximadamente: {best_rho:.5f}")


Iniciando a busca binária para encontrar o maior ρ...
------------------------------------------------------------


c:\CONTROLE_ROBUSTO\venv\Lib\site-packages\cvxpy\problems\problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Iter  1: Viável para ρ = 2.500000. Limite inferior atualizado.
Iter  2: Inviável para ρ = 3.750000. Limite superior atualizado.
Iter  3: Viável para ρ = 3.125000. Limite inferior atualizado.
Iter  4: Inviável para ρ = 3.437500. Limite superior atualizado.
Iter  5: Viável para ρ = 3.281250. Limite inferior atualizado.
Iter  6: Inviável para ρ = 3.359375. Limite superior atualizado.
Iter  7: Inviável para ρ = 3.320312. Limite superior atualizado.
Iter  8: Inviável para ρ = 3.300781. Limite superior atualizado.
Iter  9: Viável para ρ = 3.291016. Limite inferior atualizado.
Iter 10: Inviável para ρ = 3.295898. Limite superior atualizado.
Iter 11: Viável para ρ = 3.293457. Limite inferior atualizado.
Iter 12: Viável para ρ = 3.294678. Limite inferior atualizado.
Iter 13: Inviável para ρ = 3.295288. Limite superior atualizado.
Iter 14: Inviável para ρ = 3.294983. Limite superior atualizado.
Iter 15: Viável para ρ = 3.294830. Limite inferior atualizado.
Iter 16: Viável para ρ = 3.294907. Limi